In [1]:
import requests, cv2
from requests.auth import HTTPBasicAuth

In [98]:
URL = "https://suite-endpoint-api-apne2.superb-ai.com/endpoints/8f5a5eb7-3e85-416a-8c81-cdd4f085f140/inference"
ACCESS_KEY = "87Ooe3vJlV859berMYnCja8GUJpepPue2SCPwzVl"

In [82]:
IMAGE_FILE_PATH = '/home/bok/images/274.jpg'

In [83]:
image = open(IMAGE_FILE_PATH, "rb").read()

In [84]:
response = requests.post(
    url=URL,
    auth=HTTPBasicAuth("kdt2024_1-17", ACCESS_KEY),
    headers={"Content-Type": "image/jpeg"},
    data=image,
)

In [85]:
#4시작 -> 내문제, 5시작 -> 서버문제
response.status_code

200

In [86]:
print(response.json())

{'objects': [{'class': 'RASPBERRY PICO', 'score': 0.8834236860275269, 'box': [216, 299, 448, 628]}, {'class': 'USB', 'score': 0.7335350513458252, 'box': [250, 580, 307, 628]}, {'class': 'OSCILLIATOR', 'score': 0.722891092300415, 'box': [356, 404, 381, 424]}, {'class': 'BOOTSEL', 'score': 0.7168576717376709, 'box': [308, 525, 338, 566]}, {'class': 'CHIPSET', 'score': 0.7115623354911804, 'box': [307, 432, 364, 485]}, {'class': 'HOLE', 'score': 0.4126925468444824, 'box': [413, 336, 425, 349]}, {'class': 'HOLE', 'score': 0.3643624186515808, 'box': [348, 312, 364, 335]}, {'class': 'HOLE', 'score': 0.3590756058692932, 'box': [351, 312, 362, 326]}]}


In [87]:
objects = response.json()["objects"]

In [88]:
type((objects))

list

In [94]:
img = cv2.imread(IMAGE_FILE_PATH)
i = 20
name_count = 1
pre_name = None
name_dict = {}

for obj_name in objects:
    name = obj_name['class']
    if name not in name_dict:
        name_dict[name] = 1
    else:
        name_dict[name] += 1
    
for key in name_dict:
    text = f'{key} x {name_dict[key]}'
    position = (0, i) # 텍스트 시작 위치 (x, y)
    font = cv2.FONT_HERSHEY_SIMPLEX # 글꼴 설정
    font_scale = 0.5 # 글자 크기
    color_names = (255, 0, 0)
    thickness = 2 # 글자 두께
    i+=20
    cv2.putText(img, text, position, font, font_scale, color_names, thickness, cv2.LINE_AA)
    
    
# 박스 칠 좌표 설정 (예: 좌측 상단 (50, 50), 우측 하단 (200, 200))


for obj in objects:
    name = obj['class']
    start_point = (obj['box'][0],obj['box'][1])
    end_point = (obj['box'][2],obj['box'][3])
    color = (0, 255, 0) # BGR 색상 (초록색)
    thickness = 2 # 박스 선의 두께
    # 박스 그리기   
    cv2.rectangle(img, start_point, end_point, color, thickness)
    
    text = name
    position = (obj['box'][0], obj['box'][1]-20) # 텍스트 시작 위치 (x, y)
    font = cv2.FONT_HERSHEY_SIMPLEX # 글꼴 설정
    font_scale = 0.4 # 글자 크기
    color_names = (0, 255, 0)
    thickness = 1 # 글자 두께
    cv2.putText(img, text, position, font, font_scale, color_names, thickness, cv2.LINE_AA)
        
cv2.imshow('image',img)
cv2.waitKey(0)

-1

In [10]:
import os
import requests
import re  # 정규식을 사용하기 위해 import
from requests.auth import HTTPBasicAuth

count = 0
path = '/home/bok/Downloads/test_images'
# ver3
#URL = "https://suite-endpoint-api-apne2.superb-ai.com/endpoints/294d7a63-a358-4829-aebe-9562120adb3c/inference"
# ver4
URL = "https://suite-endpoint-api-apne2.superb-ai.com/endpoints/793fa5e6-4e77-4e33-bed7-b92e64596ae8/inference"
# ver5
#URL = "https://suite-endpoint-api-apne2.superb-ai.com/endpoints/b99eb756-95d8-4bd5-b147-cfb45fb2d90b/inference"
ACCESS_KEY = "87Ooe3vJlV859berMYnCja8GUJpepPue2SCPwzVl"

# 확장자가 .jpg인 파일들의 리스트를 생성
jpg_files = [f for f in os.listdir(path) if f.endswith('.jpg')]
correct = []
abnormal = 0
for images in jpg_files:
    IMAGE_FILE_PATH = path + '/' + images
    image = open(IMAGE_FILE_PATH, "rb").read()

    response = requests.post(
        url=URL,
        auth=HTTPBasicAuth("kdt2024_1-17", ACCESS_KEY),
        headers={"Content-Type": "image/jpeg"},
        data=image,
    )

    objects = response.json().get("objects", [])

    name_dict = {}

    for obj_name in objects:
        name = obj_name['class']
        if name not in name_dict:
            name_dict[name] = 1
        else:
            name_dict[name] += 1

    usb_err_cnt = 0
    boot_err_cnt = 0
    osc_err_cnt = 0
    pico_err_cnt = 0
    hole_err_cnt = 0
    chip_err_cnt = 0

    '''# 모든 조건을 다 만족할 때만 correct 리스트에 추가
    if ('BOOTSEL' in name_dict and name_dict['BOOTSEL'] == 1) and \
       ('USB' in name_dict and name_dict['USB'] == 1) and \
       ('CHIPSET' in name_dict and name_dict['CHIPSET'] == 1) and \
       ('OSCILLIATOR' in name_dict and name_dict['OSCILLIATOR'] == 1) and \
       ('RASPBERRY PICO' in name_dict and name_dict['RASPBERRY PICO'] == 1) and \
       ('HOLE' in name_dict and name_dict['HOLE'] == 4):
        count += 1
        correct.append(images)
'''

    if ('BOOTSEL' not in name_dict or name_dict['BOOTSEL'] != 1):
        boot_err_cnt += 1
    if ('USB' not in name_dict or name_dict['USB'] != 1):
        usb_err_cnt += 1

    if ('CHIPSET' not in name_dict or name_dict['CHIPSET'] != 1):
        chip_err_cnt += 1

    if ('OSCILLIATOR' not in name_dict or name_dict['OSCILLIATOR'] != 1):
        osc_err_cnt += 1

    if ('RASPBERRY PICO' not in name_dict or name_dict['RASPBERRY PICO'] != 1):
        pico_err_cnt += 1

    if ('HOLE' not in name_dict or name_dict['HOLE'] != 4):
        hole_err_cnt += 1

# 이제 correct 리스트에는 모든 조건을 만족하는 이미지 이름만 포함됩니다.
print(correct)
print(count)

print(f'boot : {boot_err_cnt}\nusb: {usb_err_cnt}\nchip : {chip_err_cnt}\nosc : {osc_err_cnt}\npico : {pico_err_cnt}\nhole : {hole_err_cnt}')


[]
0
boot : 0
usb: 0
chip : 1
osc : 0
pico : 0
hole : 0


In [16]:
import os
import requests
from requests.auth import HTTPBasicAuth

path = '/home/bok/Downloads/test_images'
# ver5
URL = "https://suite-endpoint-api-apne2.superb-ai.com/endpoints/b99eb756-95d8-4bd5-b147-cfb45fb2d90b/inference"
ACCESS_KEY = "87Ooe3vJlV859berMYnCja8GUJpepPue2SCPwzVl"

# 확장자가 .jpg인 파일들의 리스트 생성
jpg_files = [f for f in os.listdir(path) if f.endswith('.jpg')]

missing_counts = {key: 0 for key in ['RASPBERRY PICO', 'CHIPSET', 'BOOTSEL', 'USB', 'HOLE', 'OSCILLIATOR']}

for images in jpg_files:
    # 이미지 이름에서 숫자 추출
    image_number = int(''.join(filter(str.isdigit, images)))

    # 이미지 번호가 20에서 99 사이가 아니면 건너뜀
    if not (20 <= image_number <= 99):
        continue

    IMAGE_FILE_PATH = os.path.join(path, images)
    with open(IMAGE_FILE_PATH, "rb") as img_file:
        image = img_file.read()

    response = requests.post(
        url=URL,
        auth=HTTPBasicAuth("kdt2024_1-17", ACCESS_KEY),
        headers={"Content-Type": "image/jpeg"},
        data=image,
    )

    objects = response.json().get("objects", [])

    name_dict = {}
    for obj_name in objects:
        name = obj_name['class']
        name_dict[name] = name_dict.get(name, 0) + 1

    required_keys = ['RASPBERRY PICO', 'CHIPSET', 'BOOTSEL', 'USB', 'HOLE', 'OSCILLIATOR']

    # 누락된 키 확인 및 카운트 증가
    for key in required_keys:
        if key not in name_dict:
            missing_counts[key] += 1

    # 조건 확인: HOLE이 4가 아니거나 다른 객체가 1이 아닌 경우 또는 키가 누락된 경우
    if ('HOLE' not in name_dict or name_dict['HOLE'] != 4) or \
       any(name_dict.get(key, 0) != 1 for key in required_keys if key != 'HOLE'):
        print(f"Image: {images}")
        print(f"Objects: {name_dict}")
        print("-" * 50)

# 누락된 키에 대한 총 카운트 출력
print("Missing key counts:")
for key, count in missing_counts.items():
    print(f"{key}: {count}")


Image: output96.jpg
Objects: {'RASPBERRY PICO': 1, 'CHIPSET': 1, 'BOOTSEL': 1, 'USB': 1, 'HOLE': 3, 'OSCILLIATOR': 1}
--------------------------------------------------
Image: output44.jpg
Objects: {'RASPBERRY PICO': 1, 'BOOTSEL': 1, 'HOLE': 4, 'USB': 1, 'CHIPSET': 1}
--------------------------------------------------
Image: output55.jpg
Objects: {'RASPBERRY PICO': 1, 'HOLE': 4, 'CHIPSET': 1, 'USB': 1, 'BOOTSEL': 1}
--------------------------------------------------
Image: output98.jpg
Objects: {'RASPBERRY PICO': 1, 'HOLE': 4, 'USB': 1, 'OSCILLIATOR': 1, 'BOOTSEL': 1}
--------------------------------------------------
Image: output86.jpg
Objects: {'RASPBERRY PICO': 1, 'BOOTSEL': 1, 'CHIPSET': 1, 'USB': 1, 'HOLE': 3, 'OSCILLIATOR': 1}
--------------------------------------------------
Image: output33.jpg
Objects: {'USB': 1, 'RASPBERRY PICO': 1, 'CHIPSET': 1, 'HOLE': 4, 'BOOTSEL': 1}
--------------------------------------------------
Image: output50.jpg
Objects: {'RASPBERRY PICO': 1, 'US